# Plotting SFR Tracks of Galaxy Evolution

Used to plot the time evolution of the star-formation rate (SFR) for galaxies lying on the star-forming galaxy main sequence (SF GMS), or off by a certain number of dex – typically 1 dex.

In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
from datetime import datetime
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm, trange

import SiGMo as sgm

In [ ]:
project_dir = Path.cwd().parent
sfr79_dir = project_dir / 'data' / "SFR79_grids"
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.04.13-17.13.09" / "0_forward_from_z0.0729411607551602_1Gyr_dt1e-3"
# snp_dir_GMSp1_sim = project_dir / 'outputs' / '_tmp' / "2022.04.13-17.55.31" / "0_forward_from_z0.0729411607551602_1Gyr_dt1e-3_SFRoffset1.0"
# snp_dir_GMSm1_sim = project_dir / 'outputs' / '_tmp' / "2022.04.13-17.57.35" / "0_forward_from_z0.0729411607551602_1Gyr_dt1e-3_SFRoffset-1.0"

# z=2..0
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.04.19-19.30.50" / "0_forward_from_z2.0_to_z0_dt1e-3_SFRoffset0.0"
# z=1..0
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.04.19-19.39.39" / "0_forward_from_z1.0_to_z0_dt1e-3_SFRoffset0.0"

# z=2..0 with lower fgal=0.3
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.04.19-19.58.48" / "0_forward_from_z2.0_to_z0_dt1e-3_SFRoffset0.0"

# z=2..0 with higher fgal=0.5
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.04.19-20.09.21" / "0_forward_from_z2.0_to_z0_dt1e-3_SFRoffset0.0"


# on, above, below GMS, z=2...0

# on GMS
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.04.19-19.30.50" / "0_forward_from_z2.0_to_z0_dt1e-3_SFRoffset0.0"
# 1 dex above GMS
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.05.05-17.10.32" / "0_forward_from_z2.0_to_z0_dt1e-3_SFRoffset1.0"
# 1 dex below GMS
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.05.05-17.20.25" / "0_forward_from_z2.0_to_z0_dt1e-3_SFRoffset-1.0"

# on GMS, from z=6...0
snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.05.05-17.37.05" / "0_forward_from_z6.0_to_z0_dt1e-3_SFRoffset0.0"

# ATTENTION! plotting directory is based on the (on-) GMS directory parent name/name here!
plot_dir = project_dir / 'plots' / '_tmp' / snp_dir_GMS_sim.parent.name / snp_dir_GMS_sim.name   # plot dir has now same datetime name as output dir
plot_dir.mkdir(exist_ok=True, parents=True)   # create plot dir only if necessary

### Read in

In [ ]:
single_snapshots = False

# how many objects per timestep?
n_envs = 1
# this is "unknown" unless looked up someplace else, but if 'None' will be determined from some Environment snapshot
n_halos = None
n_gals = None

# reading simulation results (SDSS)
env_grid_GMS_sim, halo_grid_GMS_sim, gal_grid_GMS_sim = sgm.read_all_snapshots_from_dir(snp_dir=snp_dir_GMS_sim,  # this is where you can plug in different offsets or on-GMS runs
                                                                                        n_envs=n_envs,
                                                                                        n_halos=n_halos,
                                                                                        n_gals=n_gals,
                                                                                        single_snapshots=single_snapshots)

In [ ]:
# re-create/populate halo_grid and galaxy_grid (for FIRST Environment only) if necessary
if not single_snapshots:
    for t, env_snp in enumerate(tqdm(env_grid_GMS_sim[0])):  # HARDCODED to 1st Environment only!
        for i_halo, halo in enumerate(env_snp.data['halos']):
            halo_grid_GMS_sim[i_halo, t] = sgm.Snapshot(halo)
            for i_gal, gal in enumerate(halo['galaxies']):
                gal_grid_GMS_sim[i_halo, t] = sgm.Snapshot(gal)  # HARDCODED to 1 Galaxy per Halo!!

### Plotting

In [ ]:
# define which data to plot
gal_grid = gal_grid_GMS_sim
halo_grid = halo_grid_GMS_sim
env_grid = env_grid_GMS_sim

# define what to plot
x_type = 'mstar'
x_type_name = r'log $M_*$'
x_type_unit = r'$M_\odot$'

y_type = 'SFR'
y_type_name = r'log $SFR$'
y_type_unit = r'$M_\odot yr^{-1}$'

c_type = 'mtot'
c_type_name = r'$M_{halo}$'
c_type_unit = r'$M_\odot$'
c_snp = 0
# c_snp = -1
c_alpha = 0.75
which_objects = range(0, len(gal_grid), 1)

# grab plotting data
x_data = []
y_data = []
c_data = []
label_l = []
for i in tqdm(which_objects):
    x_data.append([np.log10(gal.data[x_type]) for gal in gal_grid[i][:]])
    y_data.append([np.log10(gal.data[y_type]) - 9. for gal in gal_grid[i][:]])
    # y_data.append([halo.data[y_type] for halo in halo_grid[i][:]])
    # if gal_grid[i][c_snp].data['mgas'] == 0 and halo_grid[i][c_snp].data['mtot'] != 0:
    #     _c_data_i = [np.nan]
    # else:
    #     _c_data_i = [halo_grid[i][c_snp].data['mtot'] / gal_grid[i][c_snp].data['mgas']]
    c_data.append([halo_grid[i][c_snp].data[c_type]])
    # c_data.append(_c_data_i)
    label_l.append(r"$\mathrm{M}_\star$" + f"= {gal_grid[i, 0].data['mstar']:.2e} \t SFR = {gal_grid[i, 0].data['SFR'] / 10 ** 9:.2e}")


# normalise the colour data to make colour the data according to it (e.g. total halo mass mtot)
c_data_range = (np.nanmin(c_data), np.nanmax(c_data))
cmap = mpl.cm.viridis_r
norm = mpl.colors.LogNorm(vmin=c_data_range[0], vmax=c_data_range[1])
mapper = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)


# grab all redshifts
z_list = [gal.data['z'] for gal in gal_grid[0][:]]


# initialise plot
fig, ax = plt.subplots(figsize=(9.5, 6), constrained_layout=True)

# plot GMS Leslie+2020
z_start = env_grid[0, 0].data['z']
z_end = env_grid[0, -1].data['z']
lbt_start = env_grid[0, 0].data['lookbacktime']
lbt_end = env_grid[0, -1].data['lookbacktime']

xlin = np.linspace(x_data[0][0], x_data[-1][0], 500)
ax.plot(xlin, sgm.GMS_Leslie2020(xlin, z=z_start, log=True), ls='-.', marker='', color='xkcd:turquoise',
        label=f"{lbt_start:.2f} Gyr ago (z={z_start:.1f})")
ax.plot(xlin, sgm.GMS_Leslie2020(xlin, z=z_end, log=True), ls=':', marker='', color='xkcd:turquoise',
        label=f"{lbt_end:.2f} Gyr ago (z={z_end:.1f})")



# plot lines at same redshift
# redshifts_plot_l = [2, 1.5, 1., 0.5, 0.]
redshifts_plot_l = [6., 5., 4., 3., 2., 1., 0.]
for redshift in redshifts_plot_l:
    z_i = sgm.find_nearest_index(z_list, redshift)
    z_line_x = [gal_x[z_i] for gal_x in x_data]
    z_line_y = [gal_y[z_i] for gal_y in y_data]
    ax.plot(z_line_x, z_line_y, ls='--', color='xkcd:magenta', alpha=0.5)


# plot actual values
for i, (x, y, c, label) in enumerate(zip(x_data, y_data, c_data, label_l)):
    c_mapped = mapper.to_rgba(c)
    c_mapped[:, -1] = c_alpha
    ax.plot(x, y, c=c_mapped, ls='-')
    # ax.plot(x[-1], y[-1], c=c_mapped, marker='o')


# add colour bar
fig.colorbar(mapper, ax=ax, label=f"{c_type_name} [{c_type_unit}] at z = {env_grid[0, c_snp].data['z']:.2f}")


# # plot one quantity like SFR79
# for i, c, label in zip(which_objects, c_data, label_l):
#     c_mapped = mapper.to_rgba(c)
#     c_mapped[:, -1] = c_alpha
#     ax[1].plot(c_data[i], SFR79_grid_xCGsim[i, -1], c=c_mapped, marker='o', label=label)


# additional fig and axes config
fig.suptitle(f'Tracing {len(which_objects)} simulated galaxies from z={z_start:.1f} to z={z_end:.1f}', fontsize=16)


ax.set_xlabel(f"{x_type_name} [{x_type_unit}]")
# ax.set_yscale('log')
ax.set_ylabel(f"{y_type_name} [{y_type_unit}]")


# this is not good, only necessary b/c of exceptionally high values
ax.set_xlim(4.5, 20)
ax.set_ylim(-5, 12)


ax.legend(title="Leslie+2020 Galaxy Main Sequence:")

# ax.text(0.95, 0.05,
#        f"Colour-coded by {c_type_name} at lookbacktime = {env_grid[0, c_snp].data['lookbacktime']:.2f} Gyr",
#        transform=ax.transAxes,
#        va='bottom', ha='right')

ax.text(0.95, 0.05,
       f"Galaxy accretion fraction: {gal_grid[0, 0].data['fgal']}",
       transform=ax.transAxes,
       va='bottom', ha='right')

In [ ]:
# save plot to disk

GMS_keyword = ''
# GMS_keyword = 'p1'
# GMS_keyword = 'm1'

fig.savefig(plot_dir / f"SFRevolution_{y_type}_vs_{x_type}_GMS{GMS_keyword}_sim_"
                       f"from_z{z_start}_to_z{z_end}_"
                       f"{datetime.now().strftime('%Y.%m.%d-%H.%M.%S')}.png")

### Plot ΔSFR offset/residuals of simulated galaxies from Leslie GMS over different redshifts

In [ ]:
# initialise plot
fig, ax = plt.subplots(figsize=(9.5, 6), constrained_layout=True)


# compute ΔSFR values
delta_sfr = []
for x_gal, y_gal in tqdm(zip(x_data, y_data), total=len(x_data)):
    delta_sfr.append([y - sgm.GMS_Leslie2020(x, z, True) for x, y, z in zip(x_gal, y_gal, z_list)])  # also take z list


# plot lines at same redshift
# redshifts_plot_l = [2, 1.5, 1., 0.5, 0.]
redshifts_plot_l = [6., 5., 4., 3., 2., 1., 0.]
for redshift in redshifts_plot_l:
    z_i = sgm.find_nearest_index(z_list, redshift)
    z_line_x = [gal_x[z_i] for gal_x in x_data]
    z_line_y = [gal_y[z_i] for gal_y in delta_sfr]
    ax.plot(z_line_x, z_line_y, ls='--', color='xkcd:magenta', alpha=0.5)


# plot actual values
for i, (x, y, c) in enumerate(zip(x_data, delta_sfr, c_data)):
    c_mapped = mapper.to_rgba(c)
    c_mapped[:, -1] = c_alpha
    ax.plot(x, y, c=c_mapped, ls='-')

# add colour bar
fig.colorbar(mapper, ax=ax, label=f"{c_type_name} [{c_type_unit}] at z = {env_grid[0, c_snp].data['z']:.2f}")


# additional fig and axes config
fig.suptitle(f'ΔSFR for {len(which_objects)} simulated galaxies from z={z_start:.1f} to z={z_end:.1f}', fontsize=16)

ax.set_xlabel(r'log $M_*$ [$M_\odot$]')
ax.set_ylabel(r'$\Delta$ SFR [dex]')

# this is not good, only necessary b/c of exceptionally high values
ax.set_xlim(4.5, 20)
ax.set_ylim(-1, 11)


In [ ]:
# save plot to disk

GMS_keyword = ''
# GMS_keyword = 'p1'
# GMS_keyword = 'm1'

fig.savefig(plot_dir / f"Delta_SFRevolution_vs_{x_type}_GMS{GMS_keyword}_sim_"
                       f"from_z{z_start}_to_z{z_end}_"
                       f"{datetime.now().strftime('%Y.%m.%d-%H.%M.%S')}.png")